In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, RandomizedSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import KBinsDiscretizer, OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.decomposition import PCA

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import BernoulliNB, ComplementNB, MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
import lightgbm as lgbm

In [ ]:
# Opción para ver todas las columnas del dataset en el notebook
pd.set_option('display.max_columns', 50)

In [ ]:
data = "data/bank-additional-full.csv"
df = pd.read_csv(data, sep=";")

In [ ]:
df.head(2)

# Práctico 04: Aprendizaje Supervisado

Para finalizar nuestro modelo, aplicaremos estrategias de sampling para dividir entre train y test y haremos crossvalidation sobre train. Realizaremos pruebas con varios clasificadores y evaluaremos los resultados con múltiples métricas. Por último calcularemos el feature importance y obtendremos conclusiones.

## Objetivo del práctico

### Train-Validation-Test
(obtener del práctico anterior)
- División del dataset en train/validation/test
- Estratificación

In [ ]:
# Reemplazamos la columna y (target) por 1 y 0
df.y = df.y.replace('yes', 1)
df.y = df.y.replace('no', 0)

In [ ]:
#Diferenciamos los atributos del target
X = df.drop(columns='y')
y = df.y

In [ ]:
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, stratify=y)   #Dejamos un conjunto de test con el 20% de los casos 
#Dado que el dataset se encuentra desbalanceado (aprox. 11% del total de casos pertenece a la clase 1), empleamos el parámetro stratified en función del target (y)
#De este modo, la muestra seleccionada a partir de la división sería representativa para las dos clases

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.2, stratify=y_temp)

In [ ]:
#Para hacer algunas pruebas sin hacer doble division
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y, test_size=0.2, stratify=y)

### Preprocesamiento

- Tratamiento de valores nulos
- Estandarización
- Encoding de variables categóricas

In [ ]:
class SelectColumnsTransformer():
    def __init__(self, columns=None):
        self.columns = columns

    def transform(self, X, **transform_params):
        cpy_df = X[self.columns].copy()
        return cpy_df

    def fit(self, X, y=None, **fit_params):
        return self

#### Pipeline genérico para pre-procesamiento

In [ ]:
#Aplicamos las transformaciones previas a los conjuntos de Train y Validation

variables_categoricas = ['job', 'marital','education','housing', 'loan','contact','poutcome']
variables_numericas = ['age', 'campaign','previous','euribor3m','cons.conf.idx' ]

# Filtramos las variables que seleccionamos
X_t = X_train[variables_categoricas + variables_numericas]
X_v = X_val[variables_categoricas + variables_numericas]     

pipeline_numerico = Pipeline([('select_numeric_columns', SelectColumnsTransformer(variables_numericas)),
                             ('standard_scaler', StandardScaler()), #Solamente activamos esta línea cuando el clasificador requiere escalado de variables numéricas (SVM, SGD)
                            ('pca', PCA(n_components=4))   #En los casos que usemos las componentes principales en lugar de las variables numéricas
                             ])

pipeline_categorico = Pipeline ([('select_categoric_columns', SelectColumnsTransformer(variables_categoricas)),
                                  ('imputer', SimpleImputer(strategy='most_frequent', missing_values="unknown")),     #Activamos en el caso que quisieramos imputar la categoría desconocido
                                 ('cat', OneHotEncoder())])

pipeline_completo = ColumnTransformer([('num', pipeline_numerico, variables_numericas),
                                       ('cat', pipeline_categorico, variables_categoricas)
                                      ])


train = pipeline_completo.fit_transform(X_t)
val = pipeline_completo.fit_transform(X_v)

### Definición de métricas

Definiremos las métricas a utilizar:
- Accuracy
- Precision
- Recall
- F1
- AUC
- PRAUC  

Además investigaremos como utilizar el classification report y confusion matrix. Adicionalmente, cómo usar crossvalidation.

Dado que el problema se encuetra desbalanceado, la Accuracy no es la medida adecuada para el análisis. Las métricas más adecuadas para el análisis comparativos de los modelos son: Precision, Recall, F1, AUC, PRAUC.

Explicación de las métricas utilizadas a un stakeholder no técnico.

El conjunto de datos con que contamos para el análisis presenta una mayor cantidad de casos pertenecientes al grupo de clientes que no contrataron el PF en la última campaña de marketing en relación a aquellos clientes que si lo contrataron, razón por la cual decimos que el dataset se encuentra desbalanceado. Esta situación implica que contamos con mayor información para caracterizar a quienes no contrarían el PF que aquella disponible para caracterizar al grupo de clientes que si contratarían el PF, es decir, aquellos que constituyen el objetivo para nosotros.

En este caso, para evaluar de manera comparativa una serie de modelos predictivos y poder juzgar cuál/es de ellos son los mejores tenemos que ser cuidadosos al momento de definir las métricas. No es recomendable confiar en métricas que se concentren únicamente en la cantidad de casos que están bien clasificados ya que se le estaría dando una mayor predominancia a las clasificaciones dentro de la clase mayoritaria (clientes que no contratan PF).

En cuanto a las métricas tenemos entonces: 
- La Precisión representa qué porcentaje de los clientes que nuestro modelo predice que van a convertir, efectivamente lo hacen.
- La Recall representa qué porcentaje de los clientes que convierten son captados correctamente por nuestro modelo predicitivo. 

Dado que nos interesa tanto la proporción de clientes que efectivamente convierten así como también la proporción de clientes que convierten bien identificados por el modelo, vamos a optar por una métrica que balancea estas dos cuestiones: la F1. Esta métrica es la media armónica de las dos anteriores, y sirve para poder evaluar comparativamente una serie de modelos contemplando los dos criterios mencionados.    

## Testeo con varios modelos

Realizaremos varios tests con diversos tipos de modelos, tanto aquellos de la librería scikit-learn, como otros que no pertencen a ella:
- Decision Tree
- SGD Classifier
- Logistic regression  
- SVM     
- Naive Bayes
- Random Forest
- XGBoost
- LightGBM

Usaremos crossvalidation y compararemos con validation y test.

Realizaremos también optimizaciones de hiperparámetros en busca de los mejores valores para las métricas, empleando tanto Grid Search como Randomizes Search (en los casos en que la búsqueda se vuelva muy compleja).

## **Modelos Analizados**

#### **MODELO BASELINE**

El modelo estimado como baseline para el problema bajo análisis fue un Árbol de Decsisiones, que fue presentado en el Práctico 3 y se incluye abajo para tomar de referencia en la comparación con los nuevos modelos que se entrenarán.

In [ ]:
#Pre-procesamiento sobre los conjuntos de Train y Test
variables_categoricas = ['job', 'marital','education','housing', 'loan','contact','poutcome']
variables_numericas = ['age', 'campaign','previous','euribor3m','cons.conf.idx' ]

# Filtramos las variables que seleccionamos
X_t = X_train[variables_categoricas + variables_numericas]
X_v = X_val[variables_categoricas + variables_numericas]


pipeline_numerico = Pipeline([
                             ('select_numeric_columns', SelectColumnsTransformer(variables_numericas))     #Para este modelo no es necesario escalar las variables numéricas
                            ])

pipeline_categorico = Pipeline ([('imputer', SimpleImputer(strategy='most_frequent', missing_values = 'unknown')),
                                   ('cat', OneHotEncoder())])

pipeline_completo = ColumnTransformer([('num', pipeline_numerico, variables_numericas),
                                       ('cat', pipeline_categorico, variables_categoricas)
                                      ])

train = pipeline_completo.fit_transform(X_t)
val = pipeline_completo.fit_transform(X_v)

In [ ]:
dt_clf = DecisionTreeClassifier(random_state=0, criterion = 'gini', max_depth = 10, min_samples_leaf = 3, min_samples_split = 2, class_weight="balanced")
dt_clf.fit(train, y_train)

In [ ]:
#Predecimos y obtenemos las métricas de este modelo
y_train_pred= dt_clf.predict(train)
y_val_pred=dt_clf.predict(val)

print("ENTRENAMIENTO")
print(classification_report(y_train, y_train_pred))

print("VALIDACIÓN")
print(classification_report(y_val, y_val_pred))

In [ ]:
#Generamos la matriz de confusión para interpretar con mayor claridad los resultados
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_val,y_val_pred)

sns.heatmap(cm, annot=True)

plt.title("Matriz de Confusión")   
plt.show()

Cabe recordar que el objetivo es maximizar los valores que se encuentran en la diagonal princial (observaciones correctamente clasificadas para cada clase) y minimizar los valores de la diagonal secundaria (errores de clasificación de cada clase). 
En este caso puntual, el objetivo sería minimizar la cantidad de observaciones en el cuadrante inferior izquierdo, que representa clientes que contrataron el PF pero fueron clasificados por el modelo como que no lo contratarían, y el cuadrante superior derecho, que representa clientes que no contrataron el PF pero fueron clasificados por el modelo como que si lo contratarían.

#### **SGDClassifier**

En una primera instancia, entrenamos este modelo a partir de la misma selección de las variables originales, mientras que en una segunda instancia, con el objetivo de mejorar las métricas, se entrenó el modelo a partir de una selección de variables categóricas y las componentes principales que habían sido obtenidas a partir del análisis de componentes principales en el Práctico 2.

In [ ]:
#Pre-procesamiento sobre los conjuntos de Train y Validation

variables_categoricas = ['job', 'marital','education','housing', 'loan','contact','poutcome']
variables_numericas = ['age', 'campaign','previous','euribor3m','cons.conf.idx' ]

# Filtramos las variables que seleccionamos
X_t = X_train[variables_categoricas + variables_numericas]
X_v = X_val[variables_categoricas + variables_numericas]     

pipeline_numerico = Pipeline([('select_numeric_columns', SelectColumnsTransformer(variables_numericas)),
                             ('standard_scaler', StandardScaler())      
                            ])

pipeline_categorico = Pipeline ([('select_categoric_columns', SelectColumnsTransformer(variables_categoricas)),
                                  ('imputer', SimpleImputer(strategy='most_frequent', missing_values="unknown")),
                                 ('cat', OneHotEncoder())])

pipeline_completo = ColumnTransformer([('num', pipeline_numerico, variables_numericas),
                                       ('cat', pipeline_categorico, variables_categoricas)
                                      ])

train = pipeline_completo.fit_transform(X_t)
val = pipeline_completo.fit_transform(X_v)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

parameters = {'model__loss':['hinge', 'log','squared_loss'], 'model__learning_rate':['constant', 'optimal', 'invscaling', 'adaptive'], 
              'model__penalty': ['l2', 'l1', 'elasticnet'], 'model__eta0': [ 1e-6,0.001, 0.01, 0.1, 1, 10]}


pipeline = Pipeline([('model',SGDClassifier( random_state= 1))])
sgd_clf = GridSearchCV(pipeline, parameters, scoring=('f1','roc_auc'), cv = 5,return_train_score=True, refit='f1')

sgd_clf.fit(train, y_train)

In [ ]:
display ('Best configuraton:')
display(sgd_clf.best_params_)
best_sgd_clf = sgd_clf.best_estimator_

In [ ]:
sgd_clf=SGDClassifier( eta0=0.1, learning_rate='constant', loss='squared_loss', penalty='l1', random_state= 1)
sgd_clf.fit(train, y_train)

In [ ]:
#Predecimos y obtenemos las métricas de este modelo
y_train_pred= sgd_clf.predict(train)
y_val_pred=sgd_clf.predict(val)

print("ENTRENAMIENTO")
print(classification_report(y_train, y_train_pred))

print("VALIDACIÓN")
print(classification_report(y_val, y_val_pred))

Como se mencionó anteriormente, se entrenó el modelo con variables categóricas y las variables obtenidas por medio del análisis de componentes principales sobre las variables numéricas. Abajo se incluye el modelo que presentó las mejores métricas.

In [ ]:
df.head(2)

In [ ]:
#Pre-procesamiento sobre los conjuntos de Train y Validation
variables_categoricas = ['job', 'marital','education','housing', 'loan','contact','poutcome']
variables_numericas = ['age', 'campaign','previous','euribor3m',"emp.var.rate","cons.price.idx","nr.employed",'cons.conf.idx' ]

# Filtramos las variables que seleccionamos
X_t = X_train[variables_categoricas + variables_numericas]
X_v = X_val[variables_categoricas + variables_numericas]     

pipeline_numerico = Pipeline([('select_numeric_columns', SelectColumnsTransformer(variables_numericas)),
                             ('standard_scaler', StandardScaler()),
                             ('pca', PCA(n_components=4))
                            ])

pipeline_categorico = Pipeline ([('select_categoric_columns', SelectColumnsTransformer(variables_categoricas)),
                                  ('imputer', SimpleImputer(strategy='most_frequent', missing_values="unknown")),
                                 ('cat', OneHotEncoder())])

pipeline_completo = ColumnTransformer([('num', pipeline_numerico, variables_numericas),
                                       ('cat', pipeline_categorico, variables_categoricas)
                                      ])

train = pipeline_completo.fit_transform(X_t)
val = pipeline_completo.fit_transform(X_v)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

parameters = {'model__loss':['hinge', 'log','squared_loss'], 'model__learning_rate':['constant', 'optimal', 'invscaling', 'adaptive'], 
              'model__penalty': ['l2', 'l1', 'elasticnet'], 'model__eta0': [ 1e-6,0.001, 0.01, 0.1, 1, 10]}


pipeline = Pipeline([('model',SGDClassifier( random_state= 1))])
clf = GridSearchCV(pipeline, parameters, scoring=('f1','roc_auc'), cv = 5,return_train_score=True, refit='f1')

sgd_clf.fit(train, y_train)

In [ ]:
display ('Best configuraton:')
display(sgd_clf.best_params_)

best_sgd_clf = sgd_clf.best_estimator_


In [ ]:
best_sgd_clf=SGDClassifier(eta0= 1, learning_rate= 'constant', loss= 'log', penalty= 'l1', random_state= 1))
best_sgd_clf.fit(train, y_train)

In [ ]:
#Predecimos y obtenemos las métricas de este modelo
y_train_pred= best_sgd_clf.predict(train)
y_val_pred=best_sgd_clf.predict(val)

print("ENTRENAMIENTO")
print(classification_report(y_train, y_train_pred))

print("VALIDACIÓN")
print(classification_report(y_val, y_val_pred))

Las métricas obtenidas a partir del modelo empleando las 4 primeras PCA mejoran, pero sigue sin ofreccer buenos resultados.

#### **Logistic Regression**

In [ ]:
#Pre-procesamiento sobre los conjuntos de Train y Validation
variables_categoricas = ['job', 'marital','education','housing', 'loan','contact','poutcome']
variables_numericas = ['age', 'campaign','previous','euribor3m','cons.conf.idx' ]

# Filtramos las variables que seleccionamos
X_t = X_train[variables_categoricas + variables_numericas]
X_v = X_val[variables_categoricas + variables_numericas]     

pipeline_numerico = Pipeline([('select_numeric_columns', SelectColumnsTransformer(variables_numericas)),
                             ('standard_scaler', StandardScaler()),
                             #('pca', PCA(n_components=4))
                            ])

pipeline_categorico = Pipeline ([('select_categoric_columns', SelectColumnsTransformer(variables_categoricas)),
                                  ('imputer', SimpleImputer(strategy='most_frequent', missing_values="unknown")),
                                 ('cat', OneHotEncoder())])

pipeline_completo = ColumnTransformer([('num', pipeline_numerico, variables_numericas),
                                       ('cat', pipeline_categorico, variables_categoricas)
                                      ])

train = pipeline_completo.fit_transform(X_t)
val = pipeline_completo.fit_transform(X_v)

In [ ]:
lr_clf=LogisticRegression(random_state=0, class_weight="balanced")
lr_clf.fit(train, y_train)

In [ ]:
#Predecimos y obtenemos las métricas de este modelo
y_train_pred= lr_clf.predict(train)
y_val_pred=lr_clf.predict(val)

print("ENTRENAMIENTO")
print(classification_report(y_train, y_train_pred))

print("VALIDACIÓN")
print(classification_report(y_val, y_val_pred))

In [ ]:
#Optimización de hiperparámetros


In [ ]:
#con PCA NO MEJORA
#Pre-procesamiento sobre los conjuntos de Train y Validation
variables_categoricas = ['job', 'marital','education','housing', 'loan','contact','poutcome']
variables_numericas = ['age', 'campaign','previous','euribor3m','cons.conf.idx' ]

# Filtramos las variables que seleccionamos
X_t = X_train[variables_categoricas + variables_numericas]
X_v = X_val[variables_categoricas + variables_numericas]     

pipeline_numerico = Pipeline([('select_numeric_columns', SelectColumnsTransformer(variables_numericas)),
                             ('standard_scaler', StandardScaler()),
                             ('pca', PCA(n_components=4))
                            ])

pipeline_categorico = Pipeline ([('select_categoric_columns', SelectColumnsTransformer(variables_categoricas)),
                                  ('imputer', SimpleImputer(strategy='most_frequent', missing_values="unknown")),
                                 ('cat', OneHotEncoder())])

pipeline_completo = ColumnTransformer([('num', pipeline_numerico, variables_numericas),
                                       ('cat', pipeline_categorico, variables_categoricas)
                                      ])

train = pipeline_completo.fit_transform(X_t)
val = pipeline_completo.fit_transform(X_v)

In [ ]:
lr_clf=LogisticRegression(random_state=0, class_weight="balanced")
lr_clf.fit(train, y_train)

In [ ]:
#Predecimos y obtenemos las métricas de este modelo
y_train_pred= lr_clf.predict(train)
y_val_pred=lr_clf.predict(val)

print("ENTRENAMIENTO")
print(classification_report(y_train, y_train_pred))

print("VALIDACIÓN")
print(classification_report(y_val, y_val_pred))

#### **SVM**

In [ ]:
#Pre-procesamiento sobre los conjuntos de Train y Validation
variables_categoricas = ['job', 'marital','education','housing', 'loan','contact','poutcome']
variables_numericas = ['age', 'campaign','previous','euribor3m','cons.conf.idx' ]

# Filtramos las variables que seleccionamos
X_t = X_train[variables_categoricas + variables_numericas]
X_v = X_val[variables_categoricas + variables_numericas]     

pipeline_numerico = Pipeline([('select_numeric_columns', SelectColumnsTransformer(variables_numericas)),
                             #('standard_scaler', StandardScaler()),
                             #('pca', PCA(n_components=4))
                            ])

pipeline_categorico = Pipeline ([('select_categoric_columns', SelectColumnsTransformer(variables_categoricas)),
                                  ('imputer', SimpleImputer(strategy='most_frequent', missing_values="unknown")),
                                 ('cat', OneHotEncoder())])

pipeline_completo = ColumnTransformer([('num', pipeline_numerico, variables_numericas),
                                       ('cat', pipeline_categorico, variables_categoricas)
                                      ])

train = pipeline_completo.fit_transform(X_t)
val = pipeline_completo.fit_transform(X_v)

In [ ]:
svc_clf=SVC(random_state=0, class_weight="balanced")
svc_clf.fit(train, y_train)

In [ ]:
#Predecimos y obtenemos las métricas de este modelo
y_train_pred= svc_clf.predict(train)
y_val_pred=svc_clf.predict(val)

print("ENTRENAMIENTO")
print(classification_report(y_train, y_train_pred))

print("VALIDACIÓN")
print(classification_report(y_val, y_val_pred))

In [ ]:
#Optimización de hiperparámetros

In [ ]:
#con PCA NO MEJORA
#Pre-procesamiento sobre los conjuntos de Train y Validation
variables_categoricas = ['job', 'marital','education','housing', 'loan','contact','poutcome']
variables_numericas = ['age', 'campaign','previous','euribor3m','cons.conf.idx' ]

# Filtramos las variables que seleccionamos
X_t = X_train[variables_categoricas + variables_numericas]
X_v = X_val[variables_categoricas + variables_numericas]     

pipeline_numerico = Pipeline([('select_numeric_columns', SelectColumnsTransformer(variables_numericas)),
                             ('standard_scaler', StandardScaler()),
                             ('pca', PCA(n_components=4))
                            ])

pipeline_categorico = Pipeline ([('select_categoric_columns', SelectColumnsTransformer(variables_categoricas)),
                                  ('imputer', SimpleImputer(strategy='most_frequent', missing_values="unknown")),
                                 ('cat', OneHotEncoder())])

pipeline_completo = ColumnTransformer([('num', pipeline_numerico, variables_numericas),
                                       ('cat', pipeline_categorico, variables_categoricas)
                                      ])

train = pipeline_completo.fit_transform(X_t)
val = pipeline_completo.fit_transform(X_v)

In [ ]:
svc_clf=SVC(random_state=0, class_weight="balanced")
svc_clf.fit(train, y_train)

In [ ]:
#Predecimos y obtenemos las métricas de este modelo
y_train_pred= svc_clf.predict(train)
y_val_pred=svc_clf.predict(val)

print("ENTRENAMIENTO")
print(classification_report(y_train, y_train_pred))

print("VALIDACIÓN")
print(classification_report(y_val, y_val_pred))

#### **Random Forest**

Al igual que en el modelo anterior, entrenamos el modelo en primera instancia con las variables originales y luego probamos contemplando las variables que surgieron del PCA.

In [ ]:
#Pre-procesamiento sobre los conjuntos de Train y Validation
variables_categoricas = ['job', 'marital','education','housing', 'loan','contact','poutcome']
variables_numericas = ['age', 'campaign','previous','euribor3m','cons.conf.idx' ]

# Filtramos las variables que seleccionamos
X_t = X_train[variables_categoricas + variables_numericas]
X_v = X_val[variables_categoricas + variables_numericas]

pipeline_numerico = Pipeline([
                             ('select_numeric_columns', SelectColumnsTransformer(variables_numericas))     #Para este modelo no es necesario escalar las variables numéricas
                            ])

pipeline_categorico = Pipeline ([('select_categoric_columns', SelectColumnsTransformer(variables_categoricas)),
                                 ('imputer', SimpleImputer(strategy='most_frequent', missing_values = None)),
                                   ('cat', OneHotEncoder())])

pipeline_completo = ColumnTransformer([('num', pipeline_numerico, variables_numericas),
                                       ('cat', pipeline_categorico, variables_categoricas)
                                      ])

train = pipeline_completo.fit_transform(X_t)
val = pipeline_completo.fit_transform(X_v)

In [ ]:
search_params = {
    'n_estimators': [100, 500, 1000],
    'criterion': ['gini', 'entropy'],
    'max_depth': [3, 6, 10, None],
    'min_samples_split': [2, 3, 4],
    'min_samples_leaf':[1,2,3]
}

forest_clf = RandomForestClassifier(random_state=42)
cv_forest_clf = GridSearchCV(forest_clf, search_params, cv=5, scoring='f1', n_jobs=-1)
cv_forest_clf.fit(train, y_train)

In [ ]:
display ('Best configuraton:')
display(cv_forest_clf.best_params_)
best_forest_clf = cv_forest_clf.best_estimator_

In [ ]:
#Predecimos y obtenemos las métricas de este modelo
y_train_pred= best_forest_clf.predict(train)
y_val_pred=best_forest_clf.predict(val)

print("ENTRENAMIENTO")
print(classification_report(y_train, y_train_pred))

print("VALIDACIÓN")
print(classification_report(y_val, y_val_pred))

Al igual que en el modelo anterior, entrenamos empleando las 4 primeras componentes que surgen del PCA sobre las variables numéricas y encontramos mejoras en las métricas.

In [ ]:
##Pre-procesamiento sobre los conjuntos de Train y Validation
variables_categoricas = ['job', 'marital','education','housing', 'loan','contact','poutcome']
variables_numericas = ['age', 'campaign','previous','euribor3m','cons.conf.idx' ]

# Filtramos las variables que seleccionamos
X_t = X_train[variables_categoricas + variables_numericas]
X_v = X_val[variables_categoricas + variables_numericas]


pipeline_numerico = Pipeline([('select_numeric_columns', SelectColumnsTransformer(variables_numericas)),
                             ('standard_scaler', StandardScaler()),
                             ('pca', PCA(n_components=4))
                            ])


pipeline_categorico = Pipeline ([('select_categoric_columns', SelectColumnsTransformer(variables_categoricas)),
                                 ('imputer', SimpleImputer(strategy='most_frequent', missing_values = None)),
                                   ('cat', OneHotEncoder())])


pipeline_completo = ColumnTransformer([('num', pipeline_numerico, variables_numericas),
                                       ('cat', pipeline_categorico, variables_categoricas)
                                      ])

train = pipeline_completo.fit_transform(X_t)
val = pipeline_completo.fit_transform(X_v)

In [ ]:
val.shape

In [ ]:
search_params = {
    'n_estimators': [100, 500, 1000],
    'criterion': ['gini', 'entropy'],
    'max_depth': [3, 6, 10, None],
    'min_samples_split': [2, 3, 4],
    'min_samples_leaf':[1,2,3]
}

forest_clf = RandomForestClassifier(random_state=42)
cv_forest_clf = GridSearchCV(forest_clf, search_params, cv=5, scoring='f1', n_jobs=-1)
cv_forest_clf.fit(train, y_train)

In [ ]:
display ('Best configuraton:')
display(cv_forest_clf.best_params_)
best_forest_clf = cv_forest_clf.best_estimator_

**Mejores parámetros encontrados para RF**, obtenidos en otra notebook. Los pegamos acá. 

'Best configuraton:'
{'criterion': 'entropy',
 'max_depth': 15,
 'min_samples_leaf': 1,
 'min_samples_split': 3,
 'n_estimators': 100}

In [ ]:
best_forest_clf=RandomForestClassifier(criterion='entropy', max_depth=15, min_samples_leaf=1, min_samples_split=3, n_estimators=100, random_state=42)
best_forest_clf.fit(train, y_train)

In [ ]:
#Predecimos y obtenemos las métricas de este modelo
y_train_pred= best_forest_clf.predict(train)
y_val_pred=best_forest_clf.predict(val)

print("ENTRENAMIENTO")
print(classification_report(y_train, y_train_pred))

print("VALIDACIÓN")
print(classification_report(y_val, y_val_pred))

#### **Decision Tree**

Repetimos el entrenamiento del Árbol de decisión a partir de las 4 primeras componentes principales y una selección de las variables categóricas, para chequear si esta opción mejora las métricas.

In [ ]:
##Pre-procesamiento sobre los conjuntos de Train y Validation
variables_categoricas = ['job', 'marital','education','housing', 'loan','contact','poutcome']
variables_numericas = ['age', 'campaign','previous','euribor3m','cons.conf.idx' ]

# Filtramos las variables que seleccionamos
X_t = X_train[variables_categoricas + variables_numericas]
X_v = X_val[variables_categoricas + variables_numericas]

pipeline_numerico = Pipeline([('select_numeric_columns', SelectColumnsTransformer(variables_numericas)),
                             ('standard_scaler', StandardScaler()),
                             ('pca', PCA(n_components=4))
                            ])

pipeline_categorico = Pipeline ([('imputer', SimpleImputer(strategy='most_frequent', missing_values = None)),
                                   ('cat', OneHotEncoder())])

pipeline_completo = ColumnTransformer([('num', pipeline_numerico, variables_numericas),
                                       ('cat', pipeline_categorico, variables_categoricas)
                                      ])

train = pipeline_completo.fit_transform(X_t)
val = pipeline_completo.fit_transform(X_v)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

parameters = {'criterion':['gini', 'entropy'], 'max_depth':[None, 3, 5, 10, 20, 40], 'min_samples_split': [2,3,4], 'min_samples_leaf':[1,2,3,4]}
tree = DecisionTreeClassifier(random_state=0)

cv_dt_clf = GridSearchCV(tree, parameters, scoring=('f1','roc_auc'), cv = 5,return_train_score=True, refit='f1')

cv_dt_clf.fit(train, y_train)

In [ ]:
display ('Best configuraton:')
display(cv_dt_clf.best_params_)
best_dt_clf = cv_dt_clf.best_estimator_

**DT Best Configurationn**, obtenido en otra notebook.

'Best configuraton:'
{'criterion': 'entropy',
 'max_depth': 10,
 'min_samples_leaf': 3,
 'min_samples_split': 2}

In [ ]:
#No me dio igual a la optimización que había hecho Santi y esta opción óptima overfitea el train
best_dt_clf=DecisionTreeClassifier(criterion='gini', max_depth=20, min_samples_leaf=1, min_samples_split=2,random_state=0)
best_dt_clf.fit(train, y_train)

In [ ]:
#Predecimos y obtenemos las métricas de este modelo
y_train_pred= best_dt_clf.predict(train)
y_val_pred=best_dt_clf.predict(val)

print("ENTRENAMIENTO")
print(classification_report(y_train, y_train_pred))

print("VALIDACIÓN")
print(classification_report(y_val, y_val_pred))

#### **Naive bayes**

In [ ]:
variables_categoricas = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'poutcome']
variables_numericas = ['age', 'campaign', 'previous', 'cons.conf.idx', 'euribor3m']

# Filtramos las variables que seleccionamos
X_t = X_train[variables_categoricas + variables_numericas]
X_v = X_val[variables_categoricas + variables_numericas]

pipeline_numerico = Pipeline([('select_numeric_columns', SelectColumnsTransformer(variables_numericas)),
                             # ('standard_scaler', StandardScaler()),
                             ("kbins_discretizer", KBinsDiscretizer(n_bins=4, encode="ordinal", strategy="uniform")),   #strategy="uniform"
                             ('bins_cat', OneHotEncoder())
                            ])

pipeline_categorico = Pipeline ([('select_categoric_columns', SelectColumnsTransformer(variables_categoricas)),
                                 #('imputer', SimpleImputer(strategy='most_frequent', missing_values="unknown")),      #podríamos no ponerlo, y que deje "desconocido" como una categoría más
                                 ('cat', OneHotEncoder())
                                 ])

pipeline_completo = ColumnTransformer([('num', pipeline_numerico, variables_numericas),
                                   ('cat', pipeline_categorico, variables_categoricas),
                                  ])

pipeline_modelo = Pipeline([('preprocess', pipeline_completo),
                            ('nb', ComplementNB())])

#The Complement Naive Bayes classifier was designed to correct the “severe assumptions” made by the standard Multinomial Naive Bayes classifier. It is particularly suited for imbalanced data sets.
#En el pre-procesamiento transformé todos los atributos en categóricos, porque es el requerimiento del tipo de modelo

In [ ]:
#Solo Pre-procesamiento
train = pipeline_completo.fit_transform(X_t)
val = pipeline_completo.transform(X_v)

In [ ]:
#Modelo más sencillo
nb=ComplementNB()
nb.fit(train, y_train)

In [ ]:
print("MÉTRICAS CONJUNTO DE TRAIN")
print(classification_report(y_train, nb.predict(train)))
print("MÉTRICAS CONJUNTO DE VALIDACIÓN")
print(classification_report(y_val, nb.predict(val)))

In [ ]:
#Grilla de parámetros para optimización
params={'alpha':[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 1.1, 1.3, 1.5],
        'fit_prior':[True, False],
        'norm':[True,False]
       }

nb=ComplementNB()

In [ ]:
#Búsqueda de los mejores parámetros
cv_nb = GridSearchCV(nb, params, scoring='f1', cv=5,refit=True,n_jobs=-1)     
cv_nb.fit(train, y_train)

In [ ]:
cv_nb.best_params_

In [ ]:
#Entrenamiento de la mejor versión encontrada del modelo
nb_best = ComplementNB(alpha=0.3, fit_prior=True, norm=True)
nb_best.fit(train, y_train)

In [ ]:
print("MÉTRICAS CONJUNTO DE TRAIN")
print(classification_report(y_train, nb_best.predict(train)))
print("MÉTRICAS CONJUNTO DE VALIDACIÓN")
print(classification_report(y_val, nb_best.predict(val)))

En la mejor especificación encontrada para este tipo de modelos, y sobre el conjunto de validación, la F1 de la clase minoritaria es 0.39 y la F1 promedio entre las dos clases es 0.65. 

Dentro de la clase minoritaria tenemos que el 36% de los clientes que el modelo indica que contratarían el PF están identificados de manera correcta, y logra capturar al 43% de los clientes que efectivamente contrarían el PF.

#### **XGBoost**

In [ ]:
import xgboost as xgb

In [ ]:
variables_categoricas = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'poutcome']
variables_numericas = ['age', 'campaign', 'previous', 'cons.conf.idx', 'euribor3m']

# Filtramos las variables que seleccionamos
X_t = X_train[variables_categoricas + variables_numericas]

pipeline_numerico = Pipeline([('select_numeric_columns', SelectColumnsTransformer(variables_numericas)),
                            ])

pipeline_categorico = Pipeline ([('select_categoric_columns', SelectColumnsTransformer(variables_categoricas)),
                                 #('imputer', SimpleImputer(strategy='most_frequent', missing_values="unknown")),      #podríamos no ponerlo, y que deje "desconocido" como una categoría más
                                 ('cat', OneHotEncoder())])

pipeline_completo = ColumnTransformer([('num', pipeline_numerico, variables_numericas),
                                   ('cat', pipeline_categorico, variables_categoricas),
                                  ])

pipeline_modelo = Pipeline([('preprocess', pipeline_completo),
                            ('xgb', xgb.XGBClassifier(seed=0))])

In [ ]:
#Solo Pre-procesamiento
train = pipeline_completo.fit_transform(X_t)
val = pipeline_completo.transform(X_v)

In [ ]:
xgb=xgb.XGBClassifier(seed=0)
xgb.fit(train, y_train)

In [ ]:
print("MÉTRICAS CONJUNTO DE TRAIN")
print(classification_report(y_train, xgb.predict(train)))
print("MÉTRICAS CONJUNTO DE VALIDACIÓN")
print(classification_report(y_val, xgb.predict(val)))

In [ ]:
#Optimización de hiperparámetros
#Grilla de parámetros
params={'objective':["binary:logistic","binary:hinge","binary:logitraw"],
        'learning_rate':[ 0.1,0.2,0.3],
        'max_depth':[2,4, 6, 7, 8, 10],
        'alpha':[2, 3, 5, 7],
        "n_estimators":[5, 7, 10]
       }
       
xg=xgb.XGBClassifier()

In [ ]:
#Búsqueda de parámetros
cv_xgb = GridSearchCV(xg, params, scoring='f1', cv=5,refit=True,n_jobs=-1)     
cv_xgb.fit(train, y_train)

In [ ]:
cv_xgb.best_params_

In [ ]:
import xgboost as xgb

In [ ]:
#Entrenamiento de la mejor versión encontrada del modelo en otra notebook
xgb_best = xgb.XGBClassifier(seed=0, alpha= 2, learning_rate= 0.1, max_depth= 7, n_estimators=7, objective='binary:hinge')
xgb_best.fit(train, y_train)

In [ ]:
print("MÉTRICAS CONJUNTO DE TRAIN")
print(classification_report(y_train, xgb_best.predict(train)))
print("MÉTRICAS CONJUNTO DE VALIDACIÓN")
print(classification_report(y_val, xgb_best.predict(val)))

En este caso, el mejor modelo encontrado sobre el conjunto de validación tiene una F1 igual a 0.48 en la clase minoritaria y una F1 promedio de las dos clases de 0.71.

Dentro de la clase minoritaria tenemos que el 44% de los clientes que el modelo indica que contratarían el PF están identificados de manera correcta, y logra capturar al 54% de los clientes que efectivamente contrarían el PF.

In [ ]:
#Probamos con una variedad de combinaciones de variables, para validar cuáles son las más apropiadas para el modelo.
#La mejor combinación de variables fue esta:
variables_categoricas = ['job', 'education', 'contact','loan', 'day_of_week', 'poutcome']
variables_numericas = ['age', 'campaign','previous', 'cons.conf.idx', 'euribor3m']

In [ ]:
# Filtramos las variables que seleccionamos
X_t = X_train[variables_categoricas + variables_numericas]
X_v = X_val[variables_categoricas + variables_numericas]

pipeline_numerico = Pipeline([('select_numeric_columns', SelectColumnsTransformer(variables_numericas)),
                            ])

pipeline_categorico = Pipeline ([('select_categoric_columns', SelectColumnsTransformer(variables_categoricas)),
                                 #('imputer', SimpleImputer(strategy='most_frequent', missing_values="unknown")),      #podríamos no ponerlo, y que deje "desconocido" como una categoría más
                                 ('cat', OneHotEncoder())])

pipeline_completo = ColumnTransformer([('num', pipeline_numerico, variables_numericas),
                                   ('cat', pipeline_categorico, variables_categoricas),
                                  ])


In [ ]:
#Solo Pre-procesamiento
train = pipeline_completo.fit_transform(X_t)
val = pipeline_completo.transform(X_v)

In [ ]:
#Optimización de hiperparámetros
#Grilla de parámetros
params={'objective':["binary:logistic","binary:hinge","binary:logitraw"],
        'learning_rate':[ 0.1,0.2,0.3],
        'max_depth':[2,4, 6, 7, 8, 10],
        'alpha':[2, 3, 5, 7],
        "n_estimators":[5, 7, 10]
       }
       
xg=xgb.XGBClassifier()

In [ ]:
#Búsqueda de parámetros
cv_xgb = GridSearchCV(xg, params, scoring='f1', cv=5,refit=True,n_jobs=-1)     
cv_xgb.fit(train, y_train)

In [ ]:
cv_xgb.best_params_

In [ ]:
#Entrenamiento de la mejor versión encontrada del modelo en otra notebook
xgb_best = xgb.XGBClassifier(seed=0, alpha= 2, learning_rate= 0.1, max_depth= 10, n_estimators=7, objective='binary:hinge')
xgb_best.fit(train, y_train)

In [ ]:
print("MÉTRICAS CONJUNTO DE TRAIN")
print(classification_report(y_train, xgb_best.predict(train)))
print("MÉTRICAS CONJUNTO DE VALIDACIÓN")
print(classification_report(y_val, xgb_best.predict(val)))

Alternativamente, probamos con optimizaciones aleatorias de parámetros y ofrecer mayor variabilidad a los parámetros.

In [ ]:
#Optimización de hiperparámetros
#Grilla de parámetros
params={'objective':["binary:logistic","binary:hinge","binary:logitraw"],
        'learning_rate':[0.05,0.1,0.15,0.2,0.25,0.3,0.4,0.5],
        'max_depth':[2,3, 4,5, 6, 7, 8, 9, 10, 12, 15],
        'alpha':[0, 0.5, 1, 2, 3, 5],
        'lambda':[0.5, 1, 2, 3, 5],
        "n_estimators":[3, 5, 6, 7, 8, 9, 10, 15],
        "booster":["gbtree","dart"],
        "gamma":[0.5,1,2,5, 7, 8],
        "tree_method":["auto","exact","approx","hist"]
       }
xg=xgb.XGBClassifier()

In [ ]:
#Búsqueda de parámetros
rcv_xgb = RandomizedSearchCV(xg, params, scoring='f1', cv=5,refit=True,n_jobs=-1)     
rcv_xgb.fit(train, y_train)

In [ ]:
rcv_xgb.best_params_

In [ ]:
#Entrenamiento de la mejor versión encontrada del modelo
xgb_best_r = xgb.XGBClassifier(seed=0, alpha= 0, booster="dart", gamma= 0.5, reg_lambda=5, learning_rate= 0.05, 
                               max_depth= 7,n_estimators=15, objective="binary:hinge", tree_method= 'hist'   
                                )   #el lambda por default es 1
xgb_best_r.fit(train, y_train)

In [ ]:
print("MÉTRICAS CONJUNTO DE TRAIN")
print(classification_report(y_train, xgb_best_r.predict(train)))
print("MÉTRICAS CONJUNTO DE VALIDACIÓN")
print(classification_report(y_val, xgb_best_r.predict(val)))

Probamos alternativamente con las primeras 4 componentes principales del PCA sobre las variables numéricas:

In [ ]:
##Pre-procesamiento sobre los conjuntos de Train y Validation
variables_categoricas = ['job', 'marital','education','housing', 'loan','contact','poutcome']
variables_numericas = ['age', 'campaign','previous','euribor3m','cons.conf.idx' ]

# Filtramos las variables que seleccionamos
X_t = X_train[variables_categoricas + variables_numericas]
X_v = X_val[variables_categoricas + variables_numericas]

pipeline_numerico = Pipeline([('select_numeric_columns', SelectColumnsTransformer(variables_numericas)),
                             ('standard_scaler', StandardScaler()),
                             ('pca', PCA(n_components=4))
                            ])

pipeline_categorico = Pipeline ([('imputer', SimpleImputer(strategy='most_frequent', missing_values = None)),
                                   ('cat', OneHotEncoder())])

pipeline_completo = ColumnTransformer([('num', pipeline_numerico, variables_numericas),
                                       ('cat', pipeline_categorico, variables_categoricas)
                                      ])

train = pipeline_completo.fit_transform(X_t)
val = pipeline_completo.fit_transform(X_v)

In [ ]:
#Optimización de hiperparámetros
#Grilla de parámetros
params={'objective':["binary:logistic","binary:hinge","binary:logitraw"],
        'learning_rate':[ 0.1,0.2,0.3],
        'max_depth':[2,4, 6, 7, 8, 10],
        'alpha':[2, 3, 5, 7],
        "n_estimators":[5, 7, 10]
       }
       
xg=xgb.XGBClassifier()

In [ ]:
#Búsqueda de parámetros
cv_xgb = GridSearchCV(xg, params, scoring='f1', cv=5,refit=True,n_jobs=-1)     
cv_xgb.fit(train, y_train)

In [ ]:
cv_xgb.best_params_

In [ ]:
#Entrenamiento de la mejor versión encontrada del modelo
xgb_best= xgb.XGBClassifier(seed=0, alpha= 5, learning_rate= 1, 
                               max_depth= 10,n_estimators=7, objective="binary:hinge"   
                                )   #el lambda por default es 1
xgb_best.fit(train, y_train)

In [ ]:
print("MÉTRICAS CONJUNTO DE TRAIN")
print(classification_report(y_train, xgb_best.predict(train)))
print("MÉTRICAS CONJUNTO DE VALIDACIÓN")
print(classification_report(y_val, xgb_best.predict(val)))

##NO PUDE REPLICAR ESTE RESULTADO

Comentario:

El mejor modelo encontrado usando como variables prpedictoras las 4 PCA, me da lo siguiente: 

MÉTRICAS CONJUNTO DE TRAIN
              precision    recall  f1-score   support

           0       0.97      0.94      0.95     22472
           1       0.57      0.72      0.63      2587
    accuracy                           0.91     25059
   macro avg       0.77      0.83      0.79     25059
weighted avg       0.93      0.91      0.92     25059

MÉTRICAS CONJUNTO DE VALIDACIÓN
              precision    recall  f1-score   support

           0       0.96      0.93      0.94      5618
           1       0.51      0.65      0.58       647
    accuracy                           0.90      6265
   macro avg       0.74      0.79      0.76      6265
weighted avg       0.91      0.90      0.91      6265

#### **LGMB**

In [ ]:
import lightgbm as lgb 

In [ ]:
variables_categoricas = ['job', 'marital','education','housing', 'loan','contact','poutcome']
variables_numericas = ['age', 'campaign','previous','euribor3m','cons.conf.idx' ]

# Filtramos las variables que seleccionamos
X_t = X_train[variables_categoricas + variables_numericas]
X_v = X_val[variables_categoricas + variables_numericas]     

pipeline_numerico = Pipeline([('select_numeric_columns', SelectColumnsTransformer(variables_numericas)),
                             ('standard_scaler', StandardScaler())      
                            ])

pipeline_categorico = Pipeline ([('select_categoric_columns', SelectColumnsTransformer(variables_categoricas)),
                                  ('imputer', SimpleImputer(strategy='most_frequent', missing_values="unknown")),
                                 ('cat', OneHotEncoder())])

pipeline_completo = ColumnTransformer([('num', pipeline_numerico, variables_numericas),
                                       ('cat', pipeline_categorico, variables_categoricas)
                                      ])


train = pipeline_completo.fit_transform(X_t)
val = pipeline_completo.fit_transform(X_v)

In [ ]:
lgb_clf = lgb.LGBMClassifier()
lgb_clf.fit(train, y_train)

In [ ]:
print("MÉTRICAS CONJUNTO DE TRAIN")
print(classification_report(y_train, lgb_clf.predict(train)))
print("MÉTRICAS CONJUNTO DE VALIDACIÓN")
print(classification_report(y_val, lgb_clf.predict(val)))

**EXPLORAR UN POCO MÁS ESTE MODELO**

In [ ]:
clf2 = lgb.LGBMClassifier(class_weight={0: 10, 
                1: 1})
clf2.fit(train, y_train)

In [ ]:
print("MÉTRICAS CONJUNTO DE TRAIN")
print(classification_report(y_train, clf2.predict(train)))
print("MÉTRICAS CONJUNTO DE VALIDACIÓN")
print(classification_report(y_val, clf2.predict(val)))

In [ ]:
#Optimización de parámetros

Probamos el entrenamiento de este modelo usando las 4 primeras componentes principales y las variables categóricas para ver si mejoran las métricas.

In [ ]:
#con PCA NO MEJORA
##Pre-procesamiento sobre los conjuntos de Train y Validation
variables_categoricas = ['job', 'marital','education','housing', 'loan','contact','poutcome']
variables_numericas = ['age', 'campaign','previous','euribor3m','cons.conf.idx' ]

# Filtramos las variables que seleccionamos
X_t = X_train[variables_categoricas + variables_numericas]
X_v = X_val[variables_categoricas + variables_numericas]

pipeline_numerico = Pipeline([('select_numeric_columns', SelectColumnsTransformer(variables_numericas)),
                             ('standard_scaler', StandardScaler()),
                             ('pca', PCA(n_components=4))
                            ])

pipeline_categorico = Pipeline ([('imputer', SimpleImputer(strategy='most_frequent', missing_values = None)),
                                   ('cat', OneHotEncoder())])

pipeline_completo = ColumnTransformer([('num', pipeline_numerico, variables_numericas),
                                       ('cat', pipeline_categorico, variables_categoricas)
                                      ])

train = pipeline_completo.fit_transform(X_t)
val = pipeline_completo.fit_transform(X_v)

In [ ]:
lgb_clf = lgb.LGBMClassifier()
lgb_clf.fit(train, y_train)

In [ ]:
print("MÉTRICAS CONJUNTO DE TRAIN")
print(classification_report(y_train, lgb_clf.predict(train)))
print("MÉTRICAS CONJUNTO DE VALIDACIÓN")
print(classification_report(y_val, lgb_clf.predict(val)))

### Feature importance y explainability

#### Decision Tree

In [ ]:
best_dt_clf = DecisionTreeClassifier(criterion='entropy', max_depth=10,
                                        min_samples_leaf=3, random_state=0)

In [ ]:
#Corremos el pipeline con los parámetros del mejor modelo econtrado:

pipeline_modelo_dt = Pipeline([('preprocess', pipeline_completo),
                            ('rf', best_dt_clf)])
pipeline_modelo_dt.fit(X_t, y_train)

#### Obtener los nombres de las variables

In [ ]:
# Si realizamos one hot encoding, vamos a tener el problema de que se incrementan el numero de features y necesitamos la nueva lista.
numeric_features = variables_pca
cat_features = pipeline_modelo_dt.named_steps['preprocess'].transformers_[1][1][1].get_feature_names(variables_categoricas)

In [ ]:
onehot_columns = np.array(cat_features)
numeric_features_list = np.array(numeric_features)
numeric_features_list = np.append(numeric_features_list, onehot_columns)

In [ ]:
# Es necesario ordenar las los valores del feature importance (utilizamos argsort para tener el orden de los indices)
sorted_idx = pipeline_modelo_dt[1].feature_importances_.argsort()
plt.figure(figsize = (18,9))
plt.barh(numeric_features_list[sorted_idx], pipeline_modelo_dt[1].feature_importances_[sorted_idx])
plt.xlabel("Decision Tree Feature Importance")
plt.show()

#### Graficar las variables originales que tuvieron más peso en el PCA

In [ ]:
pca = pipeline_completo.transformers_[0][1][2]

n_pcs= pca.components_.shape[0]
initial_feature_names = X_t[variables_numericas].columns
most_important = [np.abs(pca.components_[i]).argmax() for i in     range(n_pcs)]
most_important_names = [initial_feature_names[most_important[i]] for i in range(n_pcs)]
zipped_feats = zip(most_important_names, pipeline_modelo_dt[1].feature_importances_)
zipped_feats = sorted(zipped_feats, key=lambda x: x[1], reverse=True)
features, importances = zip(*zipped_feats)
top_features = features[:5]
top_importances = importances[:5]
plt.figure(figsize=(12,6))
plt.title('Feature Importances for PCA ')
plt.barh(range(len(top_importances)), top_importances, color='b', align='center')
plt.yticks(range(len(top_importances)), top_features)
plt.xlabel('Relative Importance in PCA')
plt.show()

In [ ]:
most_important